# Atlas activité végétation

**Référence :** https://doi.org/10.1101/040246

**Rappel :** pour enregistrer une figure, placer cette ligne après la figure en question :

<code> fig.savefig('figure.png') </code>

Générer ensuite la figure avec la cellule correspondante, puis exécuter cette ligne. La figure sera sauvegardée dans le dossier de la simulation correspondante. Vous pouvez bien sûr changer le nom ou l'extension (JPG, PDF).

In [ ]:
filename = 'resultat.nc'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import cartopy.crs as ccrs
from netCDF4 import Dataset
from scipy.special import logit, expit

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

data = Dataset(filename)

In [ ]:
longitude=data.variables['longitude'][:]
latitude=data.variables['latitude'][:]
altitude=data.variables['altitude'][:]
Time = data.variables['Time'][:]
Ls = data.variables['Ls'][:]

In [ ]:
dafirst = Time[0]
daint = Time[1] - dafirst
dalast = dafirst + (len(Time)-1)*daint
year = 0.
add = np.linspace(dafirst,dalast,num=len(Time)) ; add[0] = 0.
for iii in range(1,len(Ls)):
    if Ls[iii] - Ls[iii-1] < 0: year = year+1.
    add[iii] = year*360.
Ls_true = add + Ls

## Carte en moyenne temporelle sur la totalité de l'expérience

In [ ]:
# Paramètres utilisateurs -----------------------------------------
earthtopo = False # ajouter les traits de côte actuels
varname = 'tsurf'
vmin = 220
vmax = 350

# Code ------------------------------------------------------------
dataplt = data.variables[varname][:,:,:]

fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
if (earthtopo): ax.coastlines(resolution="110m",linewidth=1)
gl = ax.gridlines(linestyle='--',color='black',
             draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
clevs = np.linspace(vmin,vmax,29)
plt.contourf(longitude, latitude, np.mean(dataplt[:,:,:],axis=0),
             clevs, transform=ccrs.PlateCarree(),cmap="jet")
plt.title(r"Température de surface moyenne", size=14)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label(r'K',size=12,rotation=0,labelpad=15)
cb.ax.tick_params(labelsize=10)
plt.show()

## Carte en moyenne temporelle de $\chi$

In [ ]:
def psatw(temp):
  # METHOD GOFF GRATCH (HygroLP) - OVER WATER
  # -----------------------------------------
  log10ew = -7.90298*(373.16/temp-1) \
    + 5.02808 * np.log10(373.16/temp) \
    - 1.3816e-7 * (10**(11.344 * (1-temp/373.16))-1) \
    + 8.1328e-3 * (10**(-3.49149 *(373.16/temp-1))-1) \
    + np.log10(1013.246)
  return 100 * (10**(log10ew))

def psati(temp):
  # METHOD GOFF GRATCH (HygroLP) - OVER ICE
  # ---------------------------------------
  log10ei = -9.09718*(273.16/temp-1) \
    - 3.56654*np.log10(273.16/temp) \
    + 0.876793*(1-temp/273.16) \
    + np.log10(6.1071)
  return  100 * (10**(log10ei))

def leafratio(temp,pvap,alti):
    # all inputs in ISU
    vpd = (psatw(temp) - pvap)/1e3 # in kPa
    xhi=(expit(0.0545*(temp-273.15-25.)-0.5*np.log(vpd)-0.0815*alti*1e-3+1.189))
    return xhi

In [ ]:
# Paramètres utilisateurs -----------------------------------------
earthtopo = False # ajouter les traits de côte actuels
vmin = 0.
vmax = 1.

# Code ------------------------------------------------------------
psnc = data.variables['ps'][:,:,:]
tempnc = data.variables['temp'][:,:,:,:]
qvapnc = data.variables['h2o_vap'][:,:,:,:]
phisnc = data.variables['phisinit'][:,:]

pvap = qvapnc[:,0,:,:]*psnc[:,:,:]/0.622 # en Pa
alti = phisnc/9.81 # en metres

mask = alti
alti[alti <= 0] = np.nan
alti[alti > 0] = 1.

dataplt = np.zeros([len(Time),len(longitude),len(latitude)])
for itime in range(len(Time)):
    for ilon in range(len(longitude)):
        for ilat in range(len(latitude)):
            dataplt[itime,ilon,ilat] = leafratio(tempnc[itime,0,ilon,ilat],
                                                 pvap[itime,ilon,ilat],
                                                 alti[ilon,ilat])

fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
if (earthtopo): ax.coastlines(resolution="110m",linewidth=1)
gl = ax.gridlines(linestyle='--',color='black',
             draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
clevs = np.linspace(vmin,vmax,21)
plt.contourf(longitude, latitude, np.mean(dataplt[:,:,:],axis=0)*mask,
             clevs, transform=ccrs.PlateCarree(),cmap="jet")
plt.title(r"Leaf to ambient $CO_2$ partial pressures", size=14)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label(r'NU',size=12,rotation=0,labelpad=15)
cb.ax.tick_params(labelsize=10)
plt.show()